# **Probing GPT model**

Этот туториал посвящён зондированию (probing) — простому, но мощному методу для изучения внутренней работы LLM (больших языковых моделей). С его помощью можно получить приближенные знания о паттернах, которые выучивает модель и о том, как эти знания распространяются по слоям. 

### **Идея зондирования**

Подадим в модель входной текст $x$, получив прогноз $output$. Сохраним и рассмотрим цепочку скрытых состояний:

$$x \to hidden_1 \to hidden_2 \to \dots \to hidden_n \to output$$

где $hidden_i$ —  векторное представление входного из $x$ на $i$ слое. 

Если применить к $hidden_i$, собранным с определенных данных модели, можно сформировать гипотезы для ответа на вопросы:

- Есть ли в $hidden_i$ семантическая информация о частях речи? (собираем представления и решаем задачу классификации на аннотированных данных)
- Где закодированы знания о фактах или концептах, которые модель "выучила" из данных? (аналогично предыдущему)

Справедливо, что зондирование требует рутинной разметки, однако она не всегда сложна. В туториале рассмотрим:

1. Процесс зондирования на примере GPT2;
2. Анализ информативности скрытых состояний с помощью PCA;
3. Постановку эксперимента (и сам эксперимент) для ответа на вопрос: какой слой по уровню позволяет приближенно решить задачу регресси и хранит информацию по годам?;


### **Постановка задачи — где в модели хранятся знания о датах?**

Пусть у нас есть генеративная модель. Поставим вопрос: **где именно эта модель "знает", в какие годы жил тот или иной человек?**  

Для этого нам нужно:

1. Создать набор данных: пары вида (вопрос: "Когда родился Ньютон?", ответ: "1643").
2. Попустить вопросы через модель и извлекаем скрытые состояния с разных слоёв.
3. Извлечь спрогнозированную дату;
4. Обучить зонд, который предсказывает дату на основе скрытого состояния.
5. Проанализировать, на каких слоях модель наиболее эффективно хранит информацию о датах.

Приступим. 


Original: https://ai-office-hours.beehiiv.com/p/llm-probing

paper https://arxiv.org/pdf/2310.02207.pdf

In [2]:
#!pip install datasets==3.2.0 -q

In [3]:
import torch
import random
import json
import re

import numpy as np
import pandas as pd

#from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm.auto import tqdm
# suppress sklearn warning
import warnings; warnings.filterwarnings("ignore")

/Users/sabrinasadieh/Code/XAI-open_materials/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm





### **Модель GPT-2**
GPT-2 — это трансформер, состоящий из блоков (transformer layers). Для начала загрузим модель и посмотрим на пример её работы. В ходе анализа будем использовать `gpt2-medium`, обученный на `WebText`. Подробно про модель можно почитать в [статье](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf), но для себя зафиксируем, что особенности датасета, на котором обучена модель, таковы:

- Нет страниц википедии;
- Знания для модели собраны до 2019 года;
- Сбор данных проводился на основе статей на `Reddit` с фильтрацией по пользовательскому голосу >3;


Также, зафиксируем, что когда мы передаём текст в модель, происхожит следующее:

1. Текст разбивается на токены (например, `"Albert"` → `15433`, `"Einstein"` → `8372`).
2. Каждый токен превращается в векторное представление (embedding).
3. Вектора передаются через несколько слоёв трансформера
   - Последние слои добавляют всё больше информации о контексте.
4. На выходе каждого слоя получаем *последовательность* скрытых состояни1 и итоговый прогноз, которые постепенно «насыщаются» контекстом.



In [5]:
# Загружаем токенизатор и модель
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

# Установка устройства (GPU, если доступно)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# Исходный текст
text = "When was Albert Einstein Born?"
encoded_input = tokenizer(text, return_tensors='pt') #.to(device)

# Генерируем продолжение текста
with torch.no_grad():
  # = model(**encoded_input)
  output_ids = model.generate(**encoded_input, max_length=20)

# Декодируем токены обратно в текст
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(decoded_output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


When was Albert Einstein Born?

Albert Einstein was born on November 20, 1879, in


Модель *с генерацией ответа,* выолняет, в нашем случае, задачу ответа на вопрос. Постепенно доработаем код так, чтобы из ответа извлекать только год. Ограничимся годами с 1800 до 2019. 

Извлекать год будем с помощью регулярных выражений.

In [6]:
# регулярка для поиска годв

pattern = r"\b(1[89][0-9]{2}|20[01][0-9])(?:[s’']{0,2}|th)?\b"
example = "In the 2000s, I fell in love with cats."

match_ = re.search(pattern, example)

year = match_.group(1) if match_ else None

print("Extracted year:", year)

Extracted year: 2000


### **Подготовка данных и сбор hidden states**

Скрытые состояния (hidden states)— это **внутренние представления** входного текста внутри модели. Они формируются после обработки данных каждым слоем нейросети и содержат закодированную информацию о тексте. *"Что содержат скрытые состояния?"*  — открытый вопрос. Есть ряд исследований, показывающий что на разных уровнях содержится разная семантическая информация. Например, на ранних слоях — информация о частях речи и месте слова в предложении, а на поздних — значение слова. Но эта интерпретация не универсальна и в частности зондирование — попытка понимания внутренних процессов в модели.


### **Как получить скрытые состояния в Hugging Face?**
Если модель поддерживает `output_hidden_states=True`, то после обработки текста мы можем достать их так:

```python
outputs = model(**encoded_input)
hidden_states = outputs.hidden_states  # Это кортеж из N тензоров (по числу слоёв)
```

- `hidden_states[0]` — первый слой
- `hidden_states[-1]` — последний слой  
- `hidden_states[len(hidden_states) // 2]` — центральный слой  

Форма скрытых состояний:  
```
(batch_size, sequence_length, hidden_dim)
```
Например, для `gpt2-medium`:
```
(1, 6, 1024)  # 6 токенов, каждый представлен 1024-мерным вектором
```

In [7]:
tokenizer.pad_token = tokenizer.eos_token

In [8]:
# Исходный текст
texts = ["When was Albert Einstein born?", "When was Frida Kahlo born?", "When was Claus Hammel born?"]
encoded_input = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Прогоняем через модель и получаем скрытые состояния
with torch.no_grad():
    outputs = model(**encoded_input, output_hidden_states=True)
    output_ids = model.generate(**encoded_input, max_length=30)
    hidden_states = outputs.hidden_states  # Все скрытые состояния (tuple из 25 слоёв)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Посмотрим, как выглядят размеры hidden_states

In [9]:
# Получаем скрытые состояния (первый, центральный и последний слои)
first_layer = hidden_states[0]  # Первый слой
middle_layer = hidden_states[len(hidden_states) // 2]  # Центральный слой
last_layer = hidden_states[-1]  # Последний слой

# Вывод размеров скрытых состояний
print("First layer shape:", first_layer.shape)  # (batch_size, seq_len, hidden_dim)
print("Middle layer shape:", middle_layer.shape)
print("Last layer shape:", last_layer.shape)

First layer shape: torch.Size([3, 8, 1024])
Middle layer shape: torch.Size([3, 8, 1024])
Last layer shape: torch.Size([3, 8, 1024])


In [10]:
# Декодируем ответ модели
decoded_output = [tokenizer.decode(i, skip_special_tokens=True) for i in output_ids]

for i in decoded_output:
  print(i + '\n') #print("Generated text:", decoded_output)


When was Albert Einstein born?The answer is: in 1879. Einstein was born in Vienna, Austria, and died in New York City

When was Frida Kahlo born?

Frida Kahlo was born on November 28, 1894, in the small town of La Pl

When was Claus Hammel born?The answer is: in 1848. He was born in the town of St. Paul, Minnesota, and



In [11]:
# Извлечем года

pattern = r"\b(1[89][0-9]{2}|20[01][0-9])(?:[s’']{0,2}|th)?\b"

years = []

for i in decoded_output:
  match_ = re.search(pattern, i)
  year = match_.group(0) if match_ else None
  years.append(year)

print("Extracted years:", years)

Extracted years: ['1879', '1894', '1848']


In [12]:
# Выводим результаты
for text, year in zip(texts, years):
    print(f"Input: {text}")
    print(f"Generated year: {year}")
    print()


Input: When was Albert Einstein born?
Generated year: 1879

Input: When was Frida Kahlo born?
Generated year: 1894

Input: When was Claus Hammel born?
Generated year: 1848



### **Набор данных**

Будем работать с [people dataset](https://www.nature.com/articles/s41597-022-01369-4), предобработанным и очищенным для данного эксперимента. Датасет, необходимый для работы вы можете скачать с [google disc](https://drive.google.com/file/d/1QbEbJlABsbhzyKfQ6L4ES1rH3wbbuPfG/view?usp=sharing). 

In [13]:
#people_dataset = people_dataset.to_pandas() # скачать оригинальный датасет

In [14]:
#!wget

In [15]:
#!unzip 

In [16]:
most_popular = pd.read_csv('//Users/sabrinasadieh/Code/XAI-open_materials/gpt2_probing/people_dataset_prepared_most_popular.csv')

In [17]:
most_popular.head()

,name,birth,death,wiki_readers_2015_2018,birth_min,birth_max,death_min,death_max
0,Karel Matěj Čapek-Chod,1860.0,1927.0,25008,1860.0,1860.0,1927.0,1927.0
1,Florian Eichinger,1971.0,NaN,27285,1971.0,1971.0,NaN,NaN
2,Florian Jahr,1983.0,NaN,37331,1983.0,1983.0,NaN,NaN
3,Tadeusz Borowski,1922.0,1951.0,341110,1922.0,1922.0,1951.0,1951.0
4,Joseph C. O'Mahoney,1884.0,1962.0,15428,1884.0,1884.0,1962.0,1962.0


In [18]:
most_popular.shape

(556311, 8)

In [19]:
most_popular['birth'].min()

np.float64(1800.0)

И так, у нас есть набор данных, предварительно очищенный по табличке "год". В нем содержается люди, рожденные с 1800 по 2019. Нас будет интересовать две колонки — 'name' (для извлечения имени) и `birth`. 

### **Как это работает?**


1. **Выбор данных:** Мы выбираем входные данные $x$ и метки для вспомогательной задачи. Например, это может быть задача определения частей речи, извлечения временных рамок или классификации фактов.
   
2. **Извлечение скрытых состояний:** Пропускаем вход $x$ через модель и сохраняем скрытые состояния $hidden_i$ с одного или нескольких слоёв.

3. **Обучение зонда:** Мы строим простой классификатор или регрессионную модель (например, логистическую регрессию, SVM или небольшой нейросетевой слой), обучая её на $hidden_i$. Эта модель и есть наш "зонд".

4. **Анализ:** Оцениваем производительность зонда. Если зонд решает задачу хорошо, это значит, что информация, необходимая для решения задачи, закодирована в $hidden_i$.

In [20]:
# Функция для получения активаций и внимания
def get_activations_and_attention(model, enc_inputs, pattern_to_response):
    activations = {}

    with torch.no_grad():
        outputs = model(**enc_inputs, output_hidden_states=True)
        output_ids = model.generate(**enc_inputs, max_length=30)

    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    match_ = re.search(pattern_to_response, decoded_output)
    year = match_.group(1) if match_ else None

    # Извлекаем внимание (первый, средний и последний слои)
    activations['layer_1'] = outputs.hidden_states[0]  # Первый слой внимания
    activations['layer_middle'] = outputs.hidden_states[len(outputs.hidden_states) // 2]  # Средний слой внимания
    activations['layer_last'] = outputs.hidden_states[-1]  # Последний слой внимания

    return activations, year

# # Собираем данные для обучения
# activations_data_l1 = []
# activations_data_lmid = []
# activations_data_llast = []
# predicted_years = []  # Года рождения predicted
# actual_years = [] # Года рождения истинные

In [21]:
# question = "When was Albert Einstein born?"
# inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)

In [22]:
# cnt = 0
# torch.manual_seed(0)
# random.seed(0)

# np.random.seed(0)

# # Собираем активации и внимание для всех людей
# for index, row in most_popular.iloc[:5001, :].iterrows():
#     name = row['name']
#     true_year = row['birth']

#     # Создаем запросы для каждого человека
#     question = f"When was {name} born?"

#     # Токенизация
#     inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)

#     # Извлекаем активации и внимание
#     activations, predicted = get_activations_and_attention(model, inputs, pattern)

#     # Активации слоев
#     act_layer_1 = activations['layer_1'].flatten().cpu().numpy()
#     act_layer_middle = activations['layer_middle'].flatten().cpu().numpy()
#     act_layer_last = activations['layer_last'].flatten().cpu().numpy()

#     # Собираем данные
#     activations_data_l1.append(act_layer_1)
#     activations_data_lmid.append(act_layer_middle)
#     activations_data_llast.append(act_layer_last)

#     predicted_years.append(predicted)
#     actual_years.append(true_year)


#     cnt += 1

#     if cnt % 500 == 0:
#       print(cnt)


Благодарю за ваше время! Часть с EDA [здесь](https://github.com/SadSabrina/XAI-open_materials/blob/main/gpt2_probing/probing_eda_rus.ipynb). 

Присоединяйтесь к [каналу в телеграм](https://t.me/jdata_blog), чтобы видеть новые туториалы :) Также приглашаю пройти мой [rурс](https://stepik.org/a/198640) по explainable AI и, конечно, буду рада видеть в сети [LinkedIn](https://www.linkedin.com/in/sabrina-sadiekh-35181a286/).

Успешных вам проектов!
Ваш Дата-автор!